$$ \ddot{x} = \dot{v} = a $$
$$ \dot{x} = v $$

Forces of motion

$$ f = m \ddot{x} $$
$$ f_{int} + f_{ext} = m \ddot{x} $$

$f_{int}$ represents the internal forces usually defined as minimizing the potential energy of the system.

$$ f_{int} = - \nabla U $$

Given a set of positions $x$, we can compute the potential energy of the system using hooke's law.

$$ U_{ij}(x) = \frac{1}{2} k (||x_i - x_j|| - r_{ij})^2 $$

Where $r_{ij}$ is the rest length of the spring between particles $i$ and $j$.

Taking the gradient of the potential energy gives us the interal forces.

$$ \nabla U_{ij}(x_i) = k (||x_i - x_j|| - r_{ij}) \frac{(x_i - x_j)}{||x_i - x_j||} $$
$$ \nabla U_{ij}(x_i) = k (1 - \frac{r_{ij}}{||x_i - x_j||}) (x_i - x_j) $$
$$ \nabla U_{ij}(x_j) = -k (1 - \frac{r_{ij}}{||x_i - x_j||}) (x_i - x_j) $$

Now applying backwards euler integartion, we need to identify the new positions and velocities at time step $n+1$.

$$ x_{n+1} = x_n + \Delta t \dot{x}_{n+1} $$

$$ \dot{x}_{n+1} = \dot{x}_n + \Delta t \ddot{x}_{n+1} $$

$$M\ddot{x}_{n+1} = f$$
$$\ddot{x}_{n+1} = M^{-1}f$$

$$ \dot{x}_{n+1} = \dot{x}_n + \Delta t M^{-1} (f_{int,n+1} + f_{ext,n+1}) $$

By substituting we can eliminate $\dot{x}_{n+1}$

$$ x_{n+1} = x_n + \Delta t (\dot{x}_n + \Delta t M^{-1} (f_{int,n+1} + f_{ext,n+1})) $$

Moving all to one side

$$ \frac{M}{\Delta t^2} (x_{n+1} - x_n - \Delta t \dot{x}_n ) - f_{int,n+1} - f_{ext,n+1} = 0 $$

Now let's define a variable to represent the "free-flight" or inertial prediction position.

$$y = x_n + \Delta t \dot{x}_n + \Delta t^2 M^{-1} g $$

So:

$$ \frac{M}{\Delta t^2} (x_{n+1} - y) - f_{int,n+1} = 0 $$

By integrating this function using identities, we turn this into a minimization problem.

$$ \Phi(x) = \frac{1}{2\Delta t^2} (x - y)^T M (x - y) + U(x) $$

To solve this minimization problem we need to use newton's method which iteratively updates the positions from one guess $k$ to the next guess $k+1$ until convergence.

$$ x_{k+1} = x_k - H^{-1} g $$

Where $g$ is the gradient of the energy function and $H$ is the hessian of the energy function.

So we need to define the energy function's gradient, and its hessian.

$$ E(x) = \frac{1}{2\Delta t^2} (x - y)^T M (x - y) + U(x) $$
$$ \nabla E(x) = \frac{1}{\Delta t^2} M (x - y) + \nabla U(x) $$
$$ \nabla^2 E(x) = \frac{1}{\Delta t^2} M + \nabla^2 U(x) $$

We previously calculated $\nabla U(x)$. Subsituiting that in gives us:

$$ \nabla U(x_i) = k (1 - \frac{r_{ij}}{||x_i - x_j||}) (x_i - x_j) $$
$$ \nabla U(x_j) = k (1 - \frac{r_{ij}}{||x_i - x_j||}) (x_i - x_j) $$

Now let's compute the hessian of the potential energy.

$$ \nabla^2 U(x_{ii}) = k [(1 − (x_i - x_j)/||x_i - x_j||) I_3 + (L / ||x_i - x_j||^3) (x_i - x_j) (x_i - x_j)^T ] $$
$$ \nabla^2 U(x_{ii}) = \nabla^2 U(x_{jj}) $$
$$ \nabla^2 U(x_{ij}) = - \nabla^2 U(x_{ii})$$
$$ \nabla^2 U(x_{ji}) = - \nabla^2 U(x_{ii})$$

In [ ]:
import numpy as np

l = 1
k = 100
dt = 1 / 30
m = 1
gravity = np.array([0, -9.81])

# positions = np.array([[0, 0], [1, 1], [2, 2]], dtype=np.float64) # We are locking the first point in place
# velocities = np.zeros_like(positions)

# connection_matrix = np.zeros((positions.shape[0], positions.shape[0]))
# for i in range(positions.shape[0] - 1):
#     connection_matrix[i, i + 1] = 1
#     connection_matrix[i + 1, i] = 1

GRID_SIZE = 10

fixed_indicies = [0, GRID_SIZE - 1]  # Lock the two top corners in place

positions = np.zeros((GRID_SIZE*GRID_SIZE, 2))
velocities = np.zeros_like(positions)

connection_matrix = np.zeros((positions.shape[0], positions.shape[0]))

for i in range(GRID_SIZE):
    for j in range(GRID_SIZE):
        idx = i * GRID_SIZE + j
        if i != 0:
            # link to the point above
            above_idx = (i - 1) * GRID_SIZE + j
            connection_matrix[idx, above_idx] = 1
            connection_matrix[above_idx, idx] = 1
        if j != 0:
            # link to the point to the left
            left_idx = i * GRID_SIZE + (j - 1)
            connection_matrix[idx, left_idx] = 1
            connection_matrix[left_idx, idx] = 1

        if i != GRID_SIZE - 1:
            # link to the point below
            below_idx = (i + 1) * GRID_SIZE + j
            connection_matrix[idx, below_idx] = 1
            connection_matrix[below_idx, idx] = 1

        if j != GRID_SIZE - 1:
            # link to the point to the right
            right_idx = i * GRID_SIZE + (j + 1)
            connection_matrix[idx, right_idx] = 1
            connection_matrix[right_idx, idx] = 1

        positions[idx] = np.array([j, -i], dtype=np.float64)  # Arrange in a grid

dimensions = positions.shape[1]

# Given a position, we need to compute the gradient and hessian of the energy function
def hessian_block(x_i, x_j):
    d = x_i - x_j
    dist = np.linalg.norm(d)
    hess = ((k * (1 - l / dist)) * np.eye(dimensions)) + ((l / dist**3) * d @ d.T)

    return hess

def grad_block(x_i, x_j):
    d = x_i - x_j
    dist = np.linalg.norm(d)
    grad = k * (1 - (l / dist)) * d

    return grad

def get_inertial_matrix(positions, velocities, gravity):
    inertial = positions + dt * velocities + (dt**2 / m) * gravity
    return inertial
    

def grad_matrix(positions, intertial_matrix): # Returns (n, d) matrix
    n = positions.shape[0]
    grad = np.zeros((n, dimensions))

    for i in range(n):
        for j in range(n):
            if connection_matrix[i, j] == 1:
                gb = grad_block(positions[i], positions[j])
                grad[i] += gb
                grad[j] -= gb

        grad[i] += m * (positions[i] - intertial_matrix[i]) / (dt**2)

    return grad

def hess_matrix(positions): # Returns (n*d, n*d) matrix
    # $$ \nabla^2 E(x) = \frac{1}{\Delta t^2} M + \nabla^2 U(x) $$
    n = positions.shape[0]

    hess = np.zeros((n * dimensions, n * dimensions))

    for i in range(n):
        for j in range(n):
            if i < j:
                if connection_matrix[i, j] == 1:
                    hb = hessian_block(positions[i], positions[j])
                    hess[i*dimensions:(i+1)*dimensions, i*dimensions:(i+1)*dimensions] += hb
                    hess[j*dimensions:(j+1)*dimensions, j*dimensions:(j+1)*dimensions] += hb
                    hess[i*dimensions:(i+1)*dimensions, j*dimensions:(j+1)*dimensions] -= hb
                    hess[j*dimensions:(j+1)*dimensions, i*dimensions:(i+1)*dimensions] -= hb

        hess[i*dimensions:(i+1)*dimensions, i*dimensions:(i+1)*dimensions] += (m / (dt**2)) * np.eye(dimensions)

    return hess
    


# grad_block(positions[1], positions[2])
y = get_inertial_matrix(positions, velocities, gravity)
g = grad_matrix(positions, y)
h = hess_matrix(positions)
h_inv = np.linalg.inv(h)

In [ ]:
def solve(y, last_positions, iterations=None, min_delta=None):
    guess = last_positions.copy()
    i = 0
    while True:
        g = grad_matrix(guess, y)
        h = hess_matrix(guess)
        h_inv = np.linalg.inv(h)
        delta = - h_inv @ g.flatten()
        guess += delta.reshape(guess.shape)

        i += 1
        if iterations is not None and i >= iterations:
            break
        if min_delta is not None and np.linalg.norm(delta) < min_delta:
            break

    new_positions = guess
    new_velocities = (new_positions - last_positions) / dt

    return new_positions, new_velocities

new_positions, new_velocities = solve(y, positions, min_delta=1e-5)

In [ ]:
from tqdm import tqdm

TIME_STEPS = 500

point_positions = []

for t in tqdm(range(TIME_STEPS)):
    y = get_inertial_matrix(positions, velocities, gravity)
    old_positions = positions.copy()
    positions, velocities = solve(y, positions, iterations=1e-4)
    for idx in fixed_indicies:
        positions[idx] = old_positions[idx]  # Lock the fixed points in place
        velocities[idx] = np.array([0, 0])  # Lock the fixed points
    point_positions.append(positions.copy())

In [ ]:
from tqdm import tqdm

def render_simulation(positions, frame_idx=0):
    import matplotlib.pyplot as plt

    plt.figure()
    plt.title(f"Frame {frame_idx}")
    plt.xlim(-13, GRID_SIZE + 13)
    plt.ylim(- (GRID_SIZE * 2) - 5, 1)
    x = positions[:, 0]
    y = positions[:, 1]
    plt.scatter(x, y, marker='o')
    for i in range(positions.shape[0]):
        for j in range(positions.shape[0]):
            if connection_matrix[i, j] == 1:
                plt.plot([positions[i, 0], positions[j, 0]], [positions[i, 1], positions[j, 1]], color='blue')
    plt.savefig(f"output/frame_{frame_idx:04d}.png")
    plt.close()

for frame_idx, pos in tqdm(enumerate(point_positions), total=len(point_positions)):
    render_simulation(pos, frame_idx)

In [ ]:
!ffmpeg -y -framerate 30 -i output/frame_%04d.png -c:v libx264 -pix_fmt yuv420p cloth_simulation.mp4
!ffmpeg -y -i cloth_simulation.mp4 -vf "fps=15,scale=1024:-1:flags=lanczos" -loop 0 cloth_simulation.gif